In [1]:
#run only for the collab not in local
from google.colab import files
files.upload()  # Upload your kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"whitewolf18","key":"5791652d7a3c416e1d99a4ebba0fe6f4"}'}

In [2]:
#run only for the collab not in local
import os
import zipfile

os.makedirs('/root/.kaggle', exist_ok=True)

!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json


In [3]:
!kaggle datasets download iamsouravbanerjee/indian-food-images-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/iamsouravbanerjee/indian-food-images-dataset
License(s): other
 99% 351M/355M [00:00<00:00, 1.22GB/s]
100% 355M/355M [00:00<00:00, 1.22GB/s]


In [4]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
os.remove("List of Indian Foods.txt")

In [ ]:
os.rename("Indian Food Images", "dataset")

In [ ]:
DATA_DIR = "dataset/Indian Food Images"

In [ ]:
food_list = os.listdir(DATA_DIR)

In [ ]:
food_list

In [ ]:
len(food_list)

In [ ]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(300, 300))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [ ]:
def load_dataset():
    data = []
    for food in food_list:
        food_dir = os.path.join(DATA_DIR, food)
        if os.path.isdir(food_dir):
            for img_file in os.listdir(food_dir):
                img_path = os.path.join(food_dir, img_file)
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_array = load_and_preprocess_image(img_path)
                    data.append((img_array, food))
    return data

In [ ]:
data = load_dataset()

In [ ]:
def get_train_test_split(data, test_size=0.2):
    np.random.shuffle(data)
    split_index = int(len(data) * (1 - test_size))
    train_data = data[:split_index]
    test_data = data[split_index:]
    return train_data, test_data

In [ ]:
train_data, test_data = get_train_test_split(data)

In [ ]:
X_train = []
y_train = []
for item in train_data:
    img_arr, label = item
    X_train.append(img_arr[0])
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
X_test = []
y_test = []
for item in test_data:
    img_arr, label = item
    X_test.append(img_arr[0])
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_one_hot = to_categorical(y_train_encoded, num_classes=len(food_list))
y_test_one_hot = to_categorical(y_test_encoded, num_classes=len(food_list))

In [ ]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
predictions = Dense(len(food_list), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
y_train_one_hot

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train_one_hot, epochs=10, validation_split=0.2)